### Query OpenAlex for people affiliated with an organization
This notebook queries the [OpenAlex API](https://docs.openalex.org/api) via its '`/authors`' endpoint for all authors affiliated with an organization.
It takes a ROR URL as input which is used to retrieve authors with the specified ROR URL in their metadata field '`last_known_institution.ror`'. From the resulting list of people we output the ORCID iDs.

In [1]:
# Prerequisites:
import requests         # dependency to make HTTP calls

The input for the notebook is a ROR URL, e.g. '`https://ror.org/021k10z87`'

In [2]:
# input parameter
example_ror="https://ror.org/021k10z87"

We use it to query the OpenAlex API endpoint for `'authors'` with the specified ROR URL in their metadata field '`last_known_institution.ror`'.  Since the API uses [pagination](https://docs.openalex.org/api/get-lists-of-entities#pagination), we need to loop through all pages to get the complete result set.

In [3]:
# OpenAlex endpoint to query for authors
OPENALEX_API_AUTHORS = "https://api.openalex.org/authors"

# query all people that are connected to given ROR
def query_openalex_for_orga2people(ror):
    page = 1
    max_page = 1
    
    while page <= max_page:
        params = {'filter': 'last_known_institution.ror:' + ror, 'page': page}
        response = requests.get(url=OPENALEX_API_AUTHORS,
                                params=params,
                                headers= {'Accept': 'application/json'})
        response.raise_for_status()
        result=response.json()

        # calculate max page number in first loop
        if max_page == 1:
            max_page = determine_max_page(result)
        page = page + 1
        yield result

# calculate max number of result pages
def determine_max_page(response_data):
    item_count = response_data['meta']['count']
    items_per_page = response_data['meta']['per_page']
    max_page_ceil = item_count // items_per_page + bool(item_count % items_per_page)
    return max_page_ceil


#--- example execution
list_of_pages=query_openalex_for_orga2people(example_ror)

From the resulting list of people we extract and print out each ORCID iD and name.

*Note: people that do not have an ORCID within their `ids` metadata, will not be printed.*

In [4]:
# from the result pages we get from the OpenAlex API, extract the data about authors
def extract_authors_from_page(page):
    return [author for author in page.get('results') or []]

# extract ORCID from author
def extract_orcid(author):
    orcid=author.get('ids', {}).get('orcid') or ""
    orcid_id=orcid.replace("https://orcid.org/", "") if orcid else orcid
    name=author.get('display_name', "")
    return orcid_id, name


#--- example execution
for page in list_of_pages or []:
    authors=extract_authors_from_page(page)
    for author in authors or []:
        orcid, name=extract_orcid(author)
        if orcid:
            print(f"{orcid}, {name}")

0000-0002-3824-5375, Nicole Deitelhoff
0000-0002-7348-7206, Jonas Wolff
0000-0002-3536-8898, Felix Anderl
0000-0002-4259-6071, Felix S. Bethke
0000-0002-3136-0901, Thorsten Gromes
0000-0001-9698-2616, Annika Elena Poppe
0000-0002-3783-6130, Irene Weipert-Fenner
0000-0002-4793-9010, Arvid Bell
0000-0002-7012-6739, Peter Kreuzer
0000-0002-1185-8673, Daniel Mullis
0000-0001-6746-1248, Anton Peez
